In [1]:
import pandas as pd
import numpy as np

In [49]:
df_y = pd.read_csv('y_train.csv')
df_demo = pd.read_csv('demo.csv')
df_cardinfo = pd.read_csv('card_info.csv')
df_cctxn = pd.read_csv('cc_txn.csv')
df_sabal = pd.read_csv('sa_bal.csv')
df_dtxn = pd.read_csv('dtxn.csv')

all_df = {'df_y - Output for selected user ids':df_y,
        'df_demo - Personal information of customers':df_demo,
        'df_cardinfo - Credit card information':df_cardinfo,
        'df_cctxn - Credit card transaction log':df_cctxn,
        'df_sabal - Saving account balance aggregated by months':df_sabal,
        'df_dtxn - Incoming and outgoing transactions aggregated by months (exclude credit card transactions)':df_dtxn}

# all_df = [df_y, df_demo, df_cardinfo, df_cctxn, df_sabal, df_dtxn]

In [50]:
#head
for df in all_df:
    print(f"Head of DataFrame: {df} - {all_df[df].shape}:")
    display(all_df[df].head(3))
    print('\n' + '='*50 + '\n')

Head of DataFrame: df_y - Output for selected user ids - (494, 2):


,user_id,label
0,2723,1.0
1,44088,1.0
2,3139,0.0




Head of DataFrame: df_demo - Personal information of customers - (52762, 7):


,user_id,account_start_date,birth_year,gender,marital_status,family_income_segment_code,individual_income_segment_code
0,27616,1975-04-15,1961.0,F,,06,04
1,14619,1971-01-31,1943.0,F,1,04,09
2,13595,1960-11-24,1952.0,F,1,07,07




Head of DataFrame: df_cardinfo - Credit card information - (60296, 4):


...

,mcc,txn_dt,user_id,txn_amt,card_no
0,mcc_cat10,2017-04-12,17863,16391.0,21616.0
1,mcc_cat11,2017-07-12,7682,117200.0,9438.0
2,mcc_cat12,2017-05-23,33603,55602.0,39476.0




Head of DataFrame: df_sabal - Saving account balance aggregated by months - (633144, 3):


,user_id,mm,max_sa_bal
0,0,1,535052.0
1,0,10,0.0
2,0,11,161530.0




Head of DataFrame: df_dtxn - Incoming and outgoing transactions aggregated by months (exclude credit card transactions) - (490599, 4):


,user_id,mm,amt_in,amt_out
0,0,1,17725.0,4490.0
1,0,2,12431.0,29286.0
2,0,3,86274.0,0.0


In [51]:
#head
for df in all_df:
    print(f"Null check of DataFrame: {df} - {all_df[df].shape}:")

    # Clean undetectable null values
    for col in all_df[df].columns:
        if all_df[df][col].dtype == 'O':  # 'O' represents object (string) type
            all_df[df][col] = all_df[df][col].str.strip()

    all_df[df].replace('', np.nan, inplace=True)

    print(all_df[df].isnull().sum())

    print('\n' + '='*50 + '\n')

Null check of DataFrame: df_y - Output for selected user ids - (494, 2):
user_id    0
label      0
dtype: int64


Null check of DataFrame: df_demo - Personal information of customers - (52762, 7):
user_id                               0
account_start_date                    0
birth_year                            0
gender                                0
marital_status                    12295
family_income_segment_code         6822
individual_income_segment_code     3199
dtype: int64


Null check of DataFrame: df_cardinfo - Credit card information - (60296, 4):
user_id       0
bill_cyc      0
cr_lmt_amt    0
card_no       0
dtype: int64


Null check of DataFrame: df_cctxn - Credit card transaction log - (3223075, 5):
mcc        0
txn_dt     0
user_id    0
txn_amt    0
card_no    0
dtype: int64


Null check of DataFrame: df_sabal - Saving account balance aggregated by months - (633144, 3):
user_id       0
mm            0
max_sa_bal    0
dtype: int64


Null check of DataFrame: df_dtxn -

In [52]:
# for col in df_demo.columns:
#     if df_demo[col].dtype == 'O':  # 'O' represents object (string) type
#         df_demo[col] = df_demo[col].str.strip()

# df_demo.replace('', np.nan, inplace=True)

# df_demo.info()

In [60]:
df_y.sort_values('user_id', ascending=True)

,user_id,label
452,10,0.0
169,47,0.0
479,262,0.0
284,366,0.0
124,612,0.0
...,...,...
191,52039,0.0
262,52069,0.0
263,52607,0.0
424,52619,0.0


- จน userid df_y น้อยกว่า df อื่น

In [69]:
df_cctxn['txn_dt'].min()

'2017-01-26'

In [74]:
df_demo

,user_id,account_start_date,birth_year,gender,marital_status,family_income_segment_code,individual_income_segment_code
0,27616,1975-04-15,1961.0,F,NaN,06,04
1,14619,1971-01-31,1943.0,F,1,04,09
2,13595,1960-11-24,1952.0,F,1,07,07
3,31656,1972-05-25,1952.0,F,2,03,13
4,6179,1957-06-02,1939.0,M,2,01,12
...,...,...,...,...,...,...,...
52757,13557,1976-04-07,1953.0,M,1,03,NaN
52758,4293,1964-11-02,1944.0,M,1,06,03
52759,14627,1955-01-20,1930.0,M,3,06,02
52760,34018,1968-05-24,1929.0,M,1,06,04


In [75]:
df_cardinfo

,user_id,bill_cyc,cr_lmt_amt,card_no
0,0,17.0,244000,0
1,1,12.0,361000,1
2,2,12.0,385000,2
3,3,15.0,518000,3
4,4,15.0,889000,4
...,...,...,...,...
60291,52757,14.0,2146000,60291
60292,52758,12.0,1273000,60292
60293,52759,16.0,1250000,60293
60294,52760,11.0,1078000,60294
